In [ ]:
%matplotlib inline


Using rasterMath with 3d block or 2d block
==================================================================

Test notebook to validate code.



Import librairies
-------------------------------------------



In [ ]:
from museotoolbox.processing import RasterMath,image_mask_from_vector
from museotoolbox import datasets
from matplotlib import pyplot as plt
import numpy as np

Load HistoricalMap dataset
-------------------------------------------



In [ ]:
raster,vector = datasets.load_historical_data()

Initialize rasterMath with raster
------------------------------------



In [ ]:
# Set return_3d to True to have full block size (not one pixel per row)
# Create raster mask to only keep pixel inside polygons.

image_mask_from_vector(vector,raster,'/tmp/mask.tif',invert=False)

for return_3d in [True,False]:
    rM = RasterMath(raster,in_image_mask='/tmp/mask.tif',return_3d=return_3d)
    
    rM.custom_block_size(128,128) # block of 200x200pixels
    
    print(rM.get_random_block().shape)
    
    x = rM.get_random_block()
    
    # Returns with only 1 dimension
    returnFlatten = lambda x : x[...,0]
    
    # Returns 3x the original last dimension
    addOneBand = lambda x : np.repeat(x,3,axis=x.ndim-1)
    
    # Add functions to rasterMath
    rM.add_function(addOneBand,'/tmp/x_repeat_{}.tif'.format(str(return_3d)))
    rM.add_function(returnFlatten,'/tmp/x_flatten_{}.tif'.format(str(return_3d)))
    
    rM.run()
    
from osgeo import gdal
dst = gdal.Open('/tmp/x_flatten_False.tif')
arr = dst.GetRasterBand(1).ReadAsArray()
plt.imshow(np.ma.masked_where(arr == np.min(arr), arr))